In [4]:
raise Exception("Do not press Run all cells")

#### Initialize

In [1]:
import os
from _includes.ssh_methods import *
from _includes.scp_methods import *

working_folder = os.path.normpath(os.path.abspath("../bash"))
os.chdir(working_folder)
working_folder = os.getcwd() 

get_key_path = lambda: os.path.normpath(os.path.join(working_folder, relative_key_path))

In [2]:
# Hosts

# ----------------------- #

relative_key_path = 'key/ssh.pem'
full_key_path = get_key_path()

# ----------------------- #

bash_host = {
    'ip': '192.168.0.17',
    'port': '22',
    'user': 'user1',
    'key_file' : full_key_path
}

# ----------------------- #

bash_host = {
    'ip': '192.168.0.30',
    'port': '22',
    'user': 'dietpi',
    'key_file' : full_key_path
}

# ----------------------- #

docker_host = bash_host

# ----------------------- #

ansible_host = {
    'ip' : bash_host['ip'],
    'port' : '2221',
    'user' : 'ansible',
    'key_file' : full_key_path
}

#### SSH

### SCP

In [3]:
local_path="./lab1/"

In [5]:
remote_path="ansible/webserver/"

In [31]:
file = "add_disk.sh"

In [40]:
file = "install_microk8s_to_custom_path.sh"

In [ ]:
# Get file from path
fget(bash_host, remote_path + file)

In [ ]:
# Put file to user home dir
fput(bash_host, file)

In [ ]:
cmd = f'chmod +x {file}'
sh(bash_host, cmd)

In [ ]:
# Put file to path
fput(bash_host, file, remote_path)

In [ ]:
# Get folder
dget(bash_host, remote_path)

In [ ]:
# Put folder to user home dir
dput(bash_host, local_path)

In [ ]:
# Put folder to path
dput(bash_host, local_path, remote_path)

### Bash

In [ ]:
ssh(bash_host)

In [ ]:
cmd = 'git clone https://github.com/aksenov-fx/ansible.git'
sh(bash_host, cmd)

#### Files

In [ ]:
cmd = f'ls -l /usr/bin/snap'
sh(bash_host, cmd)

In [ ]:
cmd = f'ls -l {remote_path}'
sh(bash_host, cmd)

In [ ]:
cmd = f'mkdir -p {path}'
sh(bash_host, cmd)

In [ ]:
cmd = f'chmod +x {file}'
sh(bash_host, cmd)

In [42]:
cmd = f'chmod 600 {remote_path}{file}'

In [ ]:
cmd = f"chown -R ansible:ansible {remote_path}"
sush(ansible_host, cmd)

In [ ]:
cmd = f"chmod -R u+w {remote_path}"
sush(ansible_host, cmd)

In [ ]:
sush(bash_host, f'./{file}')

#### Packages

In [ ]:
cmd = f'apt-get update'
sush(bash_host, cmd)

In [ ]:
cmd = f'apt-get upgrade'
sush(bash_host, cmd)

#### System info

In [ ]:
cmd = 'cat /etc/os-release'
sh(ansible_host, cmd)

In [ ]:
cmd = f'cat /etc/group'
sh(ansible_host, cmd)

In [ ]:
cmd = f'''
    echo $PATH | tr ':' '\n'
'''
sh(bash_host, cmd)

In [ ]:
cmd = f'ps -aux --sort=%cpu'
sh(bash_host, cmd)

In [ ]:
cmd = f'ps -aux --sort=%mem'
sh(bash_host, cmd)

In [ ]:
cmd = f'df -h'
sh(bash_host, cmd)

In [ ]:
cmd = f'free -h'
sh(bash_host, cmd)

In [ ]:
cmd = f'date'
sh(bash_host, cmd)

### Docker

In [ ]:
cmd = f'curl https://get.docker.com | bash'
sh(docker_host, cmd)

In [ ]:
docker_hub_user = "aksenovfx"

#### Images

In [ ]:
image = "flask_test2"
image2 = "flask_test"
image = f"{docker_hub_user}/maventest"

In [ ]:
cmd = f'docker pull {image}'
sh(docker_host, cmd)

In [ ]:
cmd = f'''
    cd {path}
    docker build -t {image} .
'''
sh(docker_host, cmd)

In [ ]:
cmd = f'docker inspect {image}'
sh(docker_host, cmd)

#### Containers

In [ ]:
cmd = f'docker run {image}'
sh(docker_host, cmd)

In [ ]:
cmd = f'docker run --rm {image}'
sh(docker_host, cmd)

In [ ]:
cmd = f'''
    docker run --rm -d \
    -p 5432:5432 \
    -e POSTGRES_DB=db \
    -e POSTGRES_USER=user \
    -e POSTGRES_PASSWORD=password \
    {image}
'''
sh(docker_host, cmd)

In [ ]:
cmd = f'''
    docker run --rm -d \
    -p 8000:3000 \
    -e DATABASE_HOST=host.docker.internal \
    -e DATABASE_USER=user \
    -e DATABASE_PASSWORD=password \
    -e DATABASE_NAME=db \
    {image2}
'''
sh(docker_host, cmd)

In [ ]:
path_local = "/home/user1/docker-course/workbooks/05-starter-code"
path_remote = "/src/app/"
command = "java -cp /src/app/JavaApp.jar JavaApp"

cmd = f'''
    docker run --rm -v {path_local}:{path_remote} {image} {command}
'''

sh(docker_host, cmd)

In [ ]:
cmd = '''
    docker commit existing-container-name new-image-name
    docker run -d -p 8080:80 new-image-name
'''
sh(docker_host, cmd)

#### Container commands

In [ ]:
cmd = f'docker exec ubuntu-c ls -l'
sh(docker_host, cmd)

#### Compose

In [ ]:
cmd = f'''
    cd {path}
    docker compose up -d
'''
sh(docker_host, cmd)

In [ ]:
cmd = f'''
    cd {path}
    docker compose stop
'''
shc(bash_host, command)

In [ ]:
cmd = f'''
    cd {path}
    docker compose start
'''
sh(bash_host, command)

In [ ]:
cmd = f'''
    cd {path}
    docker compose down
'''
shc(bash_host, command)

#### Info

In [ ]:
cmd = f'docker ps -a'
sh(bash_host, command)

In [ ]:
cmd = f'docker logs 47f4d78a4174 --tail 10'
sh(bash_host, command)

#### Cleanup

In [ ]:
cmd = f'docker stop 8b565f60649b0aa'
shc(bash_host, command)

In [ ]:
cmd = f'''
    docker ps -q \
    --filter "ancestor=mysql/mysql-server:8.0" \
    | xargs docker stop
'''
shc(bash_host, command)

In [ ]:
cmd = f'docker rm -f 2badb31af83c'
shc(bash_host, command)

In [ ]:
cmd = f'docker rmi {image}'
shc(bash_host, command)

#### Registry

In [ ]:
cmd = f'gpg --gen-key'
sh(bash_host, command)

In [ ]:
ssh(docker_host)

#gpg --gen-key
#pass init <your_generated_gpg-id_public_key>
#docker login -u aksenovfx

In [ ]:
cmd = f'docker push {image}'
sh(bash_host, cmd)

### kubectl

#### config

In [ ]:
# install kubectl
# https://kubernetes.io/docs/tasks/tools/install-kubectl-linux/

cmd = f'''
    curl -LO https://dl.k8s.io/release/v1.32.0/bin/linux/amd64/kubectl
'''

sh(docker_host, cmd)

cmd = f'install -o root -g root -m 0755 kubectl /usr/local/bin/kubectl'
sush(docker_host, cmd)

In [ ]:
cmd = f'snap install kubectl --classic'
sush(docker_host, cmd)

#### info

In [ ]:
cmd = f'kubectl version --client'
sh(docker_host, cmd)

In [ ]:
cmd = f'kubectl config get-contexts'
sh(docker_host, cmd)

In [ ]:
cmd = f'microk8s kubectl get namespaces'
sh(docker_host, cmd)

In [ ]:
cmd = f'''
    microk8s kubectl get pods --all-namespaces
'''
sh(docker_host, cmd)

#### update

In [ ]:
#delete namespace
cmd = f'''
    microk8s kubectl delete namespace portainer
'''
shc(docker_host, cmd)

### microk8s

#### config

In [ ]:
cmd = f'''
    echo 'export PATH=$PATH:/snap/bin' >> ~/.bashrc
'''
sh(docker_host, cmd)

In [ ]:
cmd = f'cat ~/.bashrc'
sh(docker_host, cmd)

In [ ]:
cmd = 'usermod -a -G microk8s dietpi && sudo chown -R dietpi ~/.kube'
sush(docker_host, cmd)

In [ ]:
cmd = f'microk8s config > client.config'
sh(docker_host, cmd)
fget(docker_host, 'client.config')

In [ ]:
cmd = 'microk8s config > ~/.kube/config'
sh(docker_host, cmd)

In [ ]:
#enable portainer
cmd = 'apt install -y git'
sush(docker_host, cmd)

cmd = f'''
    git config --global --add safe.directory /snap/microk8s/current/addons/community/.git
    microk8s enable community
    microk8s enable portainer
'''
shc(docker_host, cmd)

In [ ]:
cmd = f'microk8s stop'
shc(docker_host, cmd)

In [ ]:
cmd = f'microk8s start'
sh(docker_host, cmd)

#### info

In [ ]:
cmd = f'microk8s status'
sh(docker_host, cmd)

### Ansible

In [ ]:
ssh(ansible_host)

In [ ]:
cmd = f'ansible -h'
sh(ansible_host, cmd)

In [ ]:
fput(ansible_host, "../ansible/webserver/ansible.cfg", remote_path)

In [ ]:
cmd = f'''
    cd {remote_path}
    ansible all -m ping
'''
sh(ansible_host, cmd)

In [ ]:
cmd = f'''
    cd {remote_path}
    ansible all -a hostname
'''
shc(ansible_host, cmd)

In [ ]:
cmd = f'''
    cd {remote_path}
    ansible all -m shell -a "hostname"
'''
shc(ansible_host, cmd)

In [ ]:
cmd = f'''
    cd {remote_path}
    ansible all -m copy -a "src=file.txt dest=dest.txt"
'''
shc(ansible_host, cmd)

In [ ]:
cmd = f'''
    cd {remote_path}
    ansible all -m service -a "name=httpd state=restarted"
'''
shc(ansible_host, cmd)

In [ ]:
cmd = f'''
    cd {remote_path}
    ansible-playbook webserver-playbook.yml --syntax-check
'''
sh(ansible_host, cmd)

In [ ]:
cmd = f'''
    cd {remote_path}
    ansible-playbook webserver-playbook.yml
'''
shc(ansible_host, cmd)

In [ ]:
cmd = f'''
    cd {remote_path}
    ansible-playbook webserver-playbook.yml
'''
shc(ansible_host, cmd)

#### Other

In [ ]:
cmd = 'systemctl enable systemd-timesyncd && sudo systemctl start systemd-timesyncd && timedatectl status'
sush(docker_host, cmd)

In [ ]:
cmd = 'lsblk -o NAME,FSTYPE,SIZE,MOUNTPOINT,LABEL'
sush(docker_host, cmd)

In [ ]:
cmd = f'shutdown -h now'
sush(docker_host, cmd)

In [ ]:
cmd = f'reboot'
sush(docker_host, cmd)

#### Temp

In [ ]:
cmd = f'rm {file}'
shc(ansible_host, cmd)